# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298411239152                   -0.85    5.2
  2   -8.300204540468       -2.75       -1.25    1.0
  3   -8.300447088198       -3.62       -1.89    3.2
  4   -8.300462156053       -4.82       -2.77    1.5
  5   -8.300464473550       -5.63       -3.07    5.1
  6   -8.300464546109       -7.14       -3.23    1.5
  7   -8.300464593902       -7.32       -3.37    5.4
  8   -8.300464619024       -7.60       -3.50    1.0
  9   -8.300464637333       -7.74       -3.70    1.1
 10   -8.300464639303       -8.71       -3.76    1.5
 11   -8.300464642550       -8.49       -3.95    1.0
 12   -8.300464643989       -8.84       -4.61    2.4


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63701594920                   -0.71    5.0
  2   -16.67749459572       -1.39       -1.14    1.4
  3   -16.67918398191       -2.77       -1.88    4.6
  4   -16.67927472767       -4.04       -2.59    4.0
  5   -16.67928583857       -4.95       -3.14    4.4
  6   -16.67928621563       -6.42       -3.53    2.1
  7   -16.67928622131       -8.25       -4.15    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31809716517                   -0.56    7.1
  2   -33.32601678727       -2.10       -1.00    1.2
  3   -33.33602058271       -2.00       -1.68    6.1
  4   -33.33683457233       -3.09       -2.29    7.0
  5   -33.33693536788       -4.00       -2.80    6.8
  6   -33.33694379409       -5.07       -3.60    2.9
  7   -33.33694391826       -6.91       -4.01    6.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298235147515                   -0.85    5.0
  2   -8.300226211795       -2.70       -1.58    1.0
  3   -8.300301120968       -4.13       -2.13    8.0
  4   -8.300337825714       -4.44       -2.20    2.6
  5   -8.300464237459       -3.90       -3.65    1.0
  6   -8.300464588883       -6.45       -3.93    4.2
  7   -8.300464641207       -7.28       -4.45    1.9


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32584987936                   -0.56    6.4
  2   -33.33174891110       -2.23       -1.28    1.0
  3   -32.30051414552   +    0.01       -1.05    8.0
  4   -32.53499990191       -0.63       -1.11    5.8
  5   -33.24211397310       -0.15       -1.28    5.8
  6   -33.28901546603       -1.33       -1.71    4.0
  7   -33.26632644956   +   -1.64       -1.64    4.4
  8   -33.33639666120       -1.15       -2.37    3.0
  9   -33.33671207946       -3.50       -2.55    2.8
 10   -33.33692210894       -3.68       -3.01    2.4
 11   -33.33694188402       -4.70       -3.68    2.6
 12   -33.33694382349       -5.71       -4.01    4.8


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.